---
title : "Overstromingsrisico scenarios"
execute:
    output: asis
---

In [ ]:
from pathlib import Path

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter

In [ ]:
path = Path.cwd() / "data_sets" / "7.flood_scenarios"
config = Config(config_path=path / "test_calculate_flood_scenario_probability.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.flood_scenarios import CalculateFloodScenarioProbability

In [ ]:
data_adapter.input("failure_probability_data")

In [ ]:
data_adapter.input("section_id_to_segment_id")

In [ ]:
calculate_flood_scenario_probability = CalculateFloodScenarioProbability(
    data_adapter=data_adapter
)

In [ ]:
calculate_flood_scenario_probability.run()